# 检索机器人

#### Step1. 读取faq数据

In [ ]:
import pandas as pd
import torch

data = pd.read_csv("./law_faq.csv")
data.head()

#### Step2. 加载模型

In [ ]:
from dual_model import DualModel

dual_model = DualModel.from_pretrained("../model/dual_model/checkpoint-500")
dual_model = dual_model.cuda()
dual_model.eval()
print("匹配模型加载成功!")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hf1/chinese-macbert-base")
tokenizer

#### Step3. 将问题编码为向量

In [ ]:
import torch
from tqdm import tqdm
questions = data["title"].tolist()
vectors = []
with torch.inference_mode():
    for i in tqdm(range(0, len(questions), 32)):
        batch_sens = questions[i:i+32]
        # print(len(batch))
        inputs = tokenizer(batch_sens, return_tensor="pt", padding=True, max_length=128, truncation=True)
        inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
        vector = dual_model.bert(**inputs)[1]
        vectors.append(vector)
vectors = torch.concat(vectors, dim=0).cpu().numpy()
vectors.shape

#### Step4. 创建索引

In [ ]:
import faiss

index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
index

#### Step5. 对问题进行向量编码

In [ ]:
question = "寻衅滋事"
with torch.inference_mode:
    inputs = tokenizer(batch_sens, return_tensor="pt", padding=True, max_length=128, truncation=True)
    inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()
q_vector.shape

#### Step6. 向量匹配(召回)

In [ ]:
faiss.normalize_L2(q_vector)
score, indexes = index.search(q_vector, 1)
topk_result = data.values[indexes[0].tolist()]
topk_result[:,0]

#### Step7. 加载交互模型(排序)

In [ ]:
from transformers import BertForSequenceClassification
corss_model = BertForSequenceClassification("../12-sentence_similaryity/cross_model/checkpoint-500/")
corss_model.eval()
print("模型加载成功")

#### Step8. 最终预测

In [ ]:
canidate = topk_result[:,0].tolist()
qes = [question] * len(canidate)
inputs = tokenizer(question, canidate, return_tensor="pt", padding=True, max_length=128, truncation=True)
inputs = {k,v.to(cross_model.device) for k,v in inputs.items()}
with torch.inference_mode():
    logits = corss_model(**inputs).logits.squeeze()
    # print(logits)
    result = torch.argmax(logits, dim=-1)
result

In [ ]:
canidate_answer = topk_result[:,1].tolist()
match_question = canidate[result.item()]
final_answer = canidate_answer[result.item()]
print(f"{question} 的相似问题是 {match_question}")